In [ ]:
#This notebook is used to create the perturbed ice models. 

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy as sp
from scipy.optimize import curve_fit
import matplotlib as mpl
import math
import random
import itertools
import copy
from scipy.stats import norm as nm
import matplotlib.mlab as mlab
import MyMultiSimTools as mst

%matplotlib inline

pgf_with_rc_fonts = {"pgf.texsystem": "pdflatex"}
mpl.rcParams.update(pgf_with_rc_fonts)

mpl.rc('font', family='Bitstream Vera Sans') 
mpl.rc('font', serif='Helvetica Neue') 
mpl.rc('text', usetex='false') 
mpl.rcParams.update({'font.size': 20})

pi = math.pi

plt.rc('text', usetex=True)
plt.rc('font', family='serif')


# Creating perturbed ice models in uncorrelated directions

In [ ]:
#Perturbing along the Diagonal

icepath = "/home/anegi/multisim/ppc/ice/spice_ftp-v2/icemodel_.dat"
central = mst.LoadIcedata(icepath)
outdir = "/data/user/anegi/ppc/models/"

amp_modes = np.arange(1,6)  
phs_modes = np.arange(1,7)

AmpShiftScales=np.linspace(-1,1,20)
PhsShiftScales=np.linspace(-1,1,20)
#For Amp 0, use scaling from [-.5,.5]

for amp_mode in amp_modes:
    DirName=outdir+"/Amp/"+str(amp_mode)
    if(not os.path.isdir(DirName)):
        os.mkdir(DirName)
    for shift in AmpShiftScales:   
        pert_amp = mst.GetIcemodel( central, 
                                    amp_modes_to_shift = [amp_mode], 
                                    amp_shifts         = [shift])
        pert=np.transpose(pert_amp)
        np.savetxt(str(DirName)+"/Amp_"+str(amp_mode)+"_"+str('%.4f'%(shift))+".dat",pert,fmt='%.6f')

for phs_mode in phs_modes:
    DirName=outdir+"/Phs/"+str(phs_mode)
    if( not os.path.isdir(DirName)):
        os.mkdir(DirName)
    for shift in PhsShiftScales:
        pert_phs = mst.GetIcemodel( central, 
                                    phs_modes_to_shift = [phs_mode], 
                                    phs_shifts         = [shift])

        pert=np.transpose(pert_phs)
        np.savetxt(str(DirName)+"/Phs_"+str(phs_mode)+"_"+str('%.4f'%(shift))+".dat",pert,fmt='%.6f')


In [ ]:
#can use the width calculated for uncorrelated modes to create perturbed icemodel with 3 sigma perterbuation along diagonal direction to get an effecient perturbation scaling and recalculate the diagonal width.

# Creating perturbed ice models in correlated directions 
Requires the Forier mode widths in correlated direction!

In [14]:
NSigmas=3
AmpSigmas=np.loadtxt("/data/user/anegi/ppc/LLHs/spice_ftp-v2/Amp_widths.txt",dtype=str)
PhsSigmas=np.loadtxt("/data/user/anegi/ppc/LLHs/spice_ftp-v2/Phs_widths.txt",dtype=str)
fcjg=(AmpSigmas[:,1])
f=fcjg.astype(float)
AmpShiftScales=(AmpSigmas[:,4].astype(float)-AmpSigmas[:,2].astype(float))
PhsShiftScales=(PhsSigmas[:,4].astype(float)-PhsSigmas[:,2].astype(float))
AmpCenters=AmpSigmas[:,2].astype(float)
PhsCenters=PhsSigmas[:,2].astype(float)
#print(AmpShiftScales,PhsShiftScales)

[0.04671078 0.04663016 0.07002907 0.00573828 0.19603149 0.17864779] [0.04548685 0.03996728 0.0656397  0.21935088 0.17374634 0.14441015]


In [13]:
#Perterbude Icemodels in correlated Amp Phs direction. 

NSigmas=3
Amp_modes=np.arange(0,5)    #[0,1,2,3,4,5]
Phs_modes=np.arange(1,6)                 

icepath = "/home/anegi/multisim/ppc/ice/spice_ftp-v2/icemodel_.dat"
outdir = "/data/user/anegi/ppc/models_3sigma/AmpPhs/"


AmpSigmas=np.loadtxt("/data/user/anegi/ppc/LLHs/spice_ftp-v2/Amp_widths.txt",dtype=str)
PhsSigmas=np.loadtxt("/data/user/anegi/ppc/LLHs/spice_ftp-v2/Phs_widths.txt",dtype=str)

#print(np.shape(AmpSigmas))
#print(len(AmpSigmas))

baseshifts = NSigmas*np.linspace(-1,1,10)

central = mst.LoadIcedata(icepath)
for mode_1 in Amp_modes:
    for mode_2 in Phs_modes:
        for i in range(0,len(AmpSigmas)):
            if AmpSigmas[i,1].astype(float)==mode_1:
                AmpShiftScale=(AmpSigmas[i,4].astype(float)-AmpSigmas[i,2].astype(float))
                AmpCenter=AmpSigmas[i,2].astype(float)
                #print(AmpCenter)
        #print(AmpCenter)
        for j in range(0,len(PhsSigmas)):                
            if PhsSigmas[j,1].astype(float)==mode_2:
                PhsShiftScale=(PhsSigmas[j,4].astype(float)-PhsSigmas[j,2].astype(float))
                PhsCenter=PhsSigmas[j,2].astype(float)
        #print(mode_1,mode_2,AmpCenter,PhsCenter) 
        
                
        shifts1=baseshifts*AmpShiftScale/np.sqrt(2.)+AmpCenter
        shifts2=baseshifts*PhsShiftScale/np.sqrt(2.)+PhsCenter
        
        for si in range(0,len(shifts1)):
            if mode_1 < mode_2:
                pert_amp_phs = mst.GetIcemodel( central, 
                                                amp_modes_to_shift = [  mode_1,mode_2 ], 
                                                amp_shifts         = [  shifts1[si], shifts2[si]  ],
                                                phs_modes_to_shift = [  mode_2 ], 
                                                phs_shifts         = [  shifts2[si] ])

                pert=np.transpose(pert_amp_phs)
                print(np.shape(pert))
                np.savetxt(str(outdir)+"/AmpPhs_"+str(mode_1)+"_"+str(mode_2)+"_"+str('%.4f'%(baseshifts[si]))+".dat",pert,fmt='%.6f')
                        
#print(shifts1)        

(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(

In [31]:
#Perturbed icemodels with AmpAmp correlation

NSigmas=3
Amp_modes=np.arange(0,5)    #[0,1,2,3,4,5]
#Phs_modes=np.arange(1,6)                 #[1]

icepath = "/home/anegi/multisim/ppc/ice/spice_ftp-v2/icemodel_.dat"
outdir = "/data/user/anegi/ppc/models_3sigma/AmpAmp/"


AmpSigmas=np.loadtxt("/data/user/anegi/ppc/LLHs/spice_ftp-v2/widths/Amp_widths.txt",dtype=str)
#PhsSigmas=np.loadtxt("/data/user/anegi/ppc/LLHs/spice_ftp-v2/widths/Phs_widths.txt",dtype=str)

#print(np.shape(AmpSigmas))
#print(len(AmpSigmas))

baseshifts = NSigmas*np.linspace(-1,1,10)

central = mst.LoadIcedata(icepath)
for mode_1 in Amp_modes:
    for mode_2 in Amp_modes:
        for i in range(0,len(AmpSigmas)):
            if AmpSigmas[i,1].astype(float)==mode_1:
                AmpShiftScale1=(AmpSigmas[i,4].astype(float)-AmpSigmas[i,2].astype(float))
                AmpCenter1=AmpSigmas[i,2].astype(float)
                #print(AmpCenter)
        #print(AmpCenter)
        for j in range(0,len(AmpSigmas)):                
            if AmpSigmas[j,1].astype(float)==mode_2:
                AmpShiftScale2=(AmpSigmas[j,4].astype(float)-AmpSigmas[j,2].astype(float))
                AmpCenter2=PhsSigmas[j,2].astype(float)
        #print(mode_1,mode_2,AmpCenter,PhsCenter) 
        
                
        shifts1=baseshifts*AmpShiftScale1/np.sqrt(2.)+AmpCenter1
        shifts2=baseshifts*AmpShiftScale2/np.sqrt(2.)+AmpCenter2
        
        for si in range(0,len(shifts1)):
            if mode_1 < mode_2:
                pert_amp_amp = mst.GetIcemodel( central, 
                                                amp_modes_to_shift = [  mode_1,mode_2 ], 
                                                amp_shifts         = [  shifts1[si], shifts2[si]  ])

                pert=np.transpose(pert_amp_amp)
                #print(np.shape(pert))
                np.savetxt(str(outdir)+"/AmpAmp_"+str(mode_1)+"_"+str(mode_2)+"_"+str('%.4f'%(baseshifts[si]))+".dat",pert,fmt='%.6f')
                        
#print(shifts1)        

In [34]:
#Perturbed icemodels with PhsPhs correlation

NSigmas=3
#Phs_modes=np.arange(0,5)    #[0,1,2,3,4,5]
Phs_modes=np.arange(1,6)                 #[1]

icepath = "/home/anegi/multisim/ppc/ice/spice_ftp-v2/icemodel_.dat"
outdir = "/data/user/anegi/ppc/models_3sigma/PhsPhs/"


#PhsSigmas=np.loadtxt("/data/user/anegi/ppc/LLHs/spice_ftp-v2/Phs_widths.txt",dtype=str)
PhsSigmas=np.loadtxt("/data/user/anegi/ppc/LLHs/spice_ftp-v2/Phs_widths.txt",dtype=str)

#print(np.shape(PhsSigmas))
#print(len(PhsSigmas))

baseshifts = NSigmas*np.linspace(-1,1,10)

central = mst.LoadIcedata(icepath)
for mode_1 in Phs_modes:
    for mode_2 in Phs_modes:
        for i in range(0,len(PhsSigmas)):
            if PhsSigmas[i,1].astype(float)==mode_1:
                PhsShiftScale1=(PhsSigmas[i,4].astype(float)-PhsSigmas[i,2].astype(float))
                PhsCenter1=PhsSigmas[i,2].astype(float)
                #print(PhsCenter)
        #print(PhsCenter)
        for j in range(0,len(PhsSigmas)):                
            if PhsSigmas[j,1].astype(float)==mode_2:
                PhsShiftScale2=(PhsSigmas[j,4].astype(float)-PhsSigmas[j,2].astype(float))
                PhsCenter2=PhsSigmas[j,2].astype(float)
        #print(mode_1,mode_2,PhsCenter,PhsCenter) 
        
                
        shifts1=baseshifts*PhsShiftScale1/np.sqrt(2.)+PhsCenter1
        shifts2=baseshifts*PhsShiftScale2/np.sqrt(2.)+PhsCenter2
        
        for si in range(0,len(shifts1)):
            if mode_1 < mode_2:
                pert_Phs_phs = mst.GetIcemodel( central, 
                                                phs_modes_to_shift = [  mode_1,mode_2 ], 
                                                phs_shifts         = [  shifts1[si], shifts2[si]  ])


                pert=np.transpose(pert_Phs_phs)
                print(np.shape(pert))
                np.savetxt(str(outdir)+"/PhsPhs_"+str(mode_1)+"_"+str(mode_2)+"_"+str('%.4f'%(baseshifts[si]))+".dat",pert,fmt='%.6f')
                        
#print(shifts1)        

(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
(171, 7)
